In [1]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Debug 
import ipdb
deb = ipdb.set_trace

# General imports
import os
import sys
import yaml
import cortex
import numpy as np
import pandas as pd

# Personal imports
sys.path.append("{}/../../analysis_code/utils".format(os.getcwd()))
from settings_utils import load_settings
from surface_utils import load_surface ,make_surface_image
from pycortex_utils import load_surface_pycortex, set_pycortex_config_file, draw_cortex
from cifti_utils import from_91k_to_32k, from_170k_to_59k

In [2]:
# Directories
main_dir = '/Users/uriel/disks/meso_shared'
project_dir = 'RetinoMaps'

In [3]:
# Load settings
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../../"))
settings_path = os.path.join(base_dir, project_dir, "settings.yml")
prf_settings_path = os.path.join(base_dir, project_dir, "prf-analysis.yml")
settings = load_settings([settings_path, prf_settings_path])
analysis_info = settings[0]


rois_group_mmp = analysis_info['rois-group-mmp']

In [4]:
format_ = '32k'
full_brain_format = '91k'
subject = 'sub-hcp{}'.format(format_)


In [5]:
# Load rois mmp mask
roi_dir = '{}/{}/derivatives/pp_data/cortex/db/{}/rois'.format(main_dir, project_dir, subject)
roi_mmp_brain_npz_fn = '{}/{}_rois-mmp.npz'.format(roi_dir, format_)
roi_mmp_lh_npz_fn = '{}/{}_hemi-L_rois-mmp.npz'.format(roi_dir, format_)
roi_mmp_rh_npz_fn = '{}/{}_hemi-R_rois-mmp.npz'.format(roi_dir, format_)

rois_mmp_masks_brain = dict(np.load(roi_mmp_brain_npz_fn))
rois_mmp_masks_lh = dict(np.load(roi_mmp_lh_npz_fn))
rois_mmp_masks_rh = dict(np.load(roi_mmp_rh_npz_fn))


In [6]:
# Load ful brain mask to extend rois masks 
full_brain_to_cortex_mask_fn = '{}/{}/derivatives/pp_data/cortex/db/{}/masks/{}_cortex_mask.npz'.format(main_dir, project_dir, subject, full_brain_format)
full_brain_to_cortex_mask = dict(np.load(full_brain_to_cortex_mask_fn))['brain_mask']

In [7]:
# make full brain masks
rois_mmp_masks_full_lh_dict = {}
rois_mmp_masks_full_rh_dict = {}
rois_mmp_masks_full_brain_dict = {}

cortex_idx = np.where(full_brain_to_cortex_mask)[0]

n_vert_lh = next(iter(rois_mmp_masks_lh.values())).shape[0]
n_vert_rh = next(iter(rois_mmp_masks_rh.values())).shape[0]

lh_cortex_idx = cortex_idx[:n_vert_lh]
rh_cortex_idx = cortex_idx[n_vert_lh:n_vert_lh + n_vert_rh]

# --- Left hemisphere ---
for roi_name, roi_mask in rois_mmp_masks_lh.items():
    roi_mask_full_lh = np.zeros_like(full_brain_to_cortex_mask, dtype=bool)
    roi_mask_full_lh[lh_cortex_idx] = roi_mask
    rois_mmp_masks_full_lh_dict[roi_name] = roi_mask_full_lh

# --- Right hemisphere ---
for roi_name, roi_mask in rois_mmp_masks_rh.items():
    roi_mask_full_rh = np.zeros_like(full_brain_to_cortex_mask, dtype=bool)
    roi_mask_full_rh[rh_cortex_idx] = roi_mask
    rois_mmp_masks_full_rh_dict[roi_name] = roi_mask_full_rh

# --- Full brain ROIs ---
for roi_name, roi_mask in rois_mmp_masks_brain.items():
    roi_mask_full_brain = np.zeros_like(full_brain_to_cortex_mask, dtype=bool)
    roi_mask_full_brain[cortex_idx] = roi_mask
    rois_mmp_masks_full_brain_dict[roi_name] = roi_mask_full_brain

In [8]:
# Export masks as npz
np.savez('{}/{}_hemi-L_rois-mmp.npz'.format(roi_dir, full_brain_format), **rois_mmp_masks_full_lh_dict)
np.savez('{}/{}_hemi-R_rois-mmp.npz'.format(roi_dir, full_brain_format), **rois_mmp_masks_full_rh_dict)
np.savez('{}/{}_rois-mmp.npz'.format(roi_dir, full_brain_format), **rois_mmp_masks_full_brain_dict)

In [9]:
# make rois-mmp-group dict 
rois_group_mmp_masks_lh = {}
rois_group_mmp_masks_rh = {}
rois_group_mmp_masks_brain = {}

for group_name, roi_list in rois_group_mmp.items():
    masks_lh = [rois_mmp_masks_lh[roi] for roi in roi_list if roi in rois_mmp_masks_lh]
    masks_rh = [rois_mmp_masks_rh[roi] for roi in roi_list if roi in rois_mmp_masks_rh]
    masks_brain = [rois_mmp_masks_brain[roi] for roi in roi_list if roi in rois_mmp_masks_brain]

    if len(masks_lh) > 0:
        rois_group_mmp_masks_lh[group_name] = np.logical_or.reduce(masks_lh)

    if len(masks_rh) > 0:
        rois_group_mmp_masks_rh[group_name] = np.logical_or.reduce(masks_rh)
        
    if len(masks_brain) > 0:
        rois_group_mmp_masks_brain[group_name] = np.logical_or.reduce(masks_brain)

In [10]:
# make full brain masks
rois_group_mmp_masks_full_lh_dict = {}
rois_group_mmp_masks_full_rh_dict = {}
rois_group_mmp_masks_full_brain_dict = {}

cortex_idx = np.where(full_brain_to_cortex_mask)[0]

n_vert_lh = next(iter(rois_mmp_masks_lh.values())).shape[0]
n_vert_rh = next(iter(rois_mmp_masks_rh.values())).shape[0]

lh_cortex_idx = cortex_idx[:n_vert_lh]
rh_cortex_idx = cortex_idx[n_vert_lh:n_vert_lh + n_vert_rh]

# --- Left hemisphere ---
for roi_name, roi_mask in rois_group_mmp_masks_lh.items():
    roi_mask_full_lh = np.zeros_like(full_brain_to_cortex_mask, dtype=bool)
    roi_mask_full_lh[lh_cortex_idx] = roi_mask
    rois_group_mmp_masks_full_lh_dict[roi_name] = roi_mask_full_lh

# --- Right hemisphere ---
for roi_name, roi_mask in rois_group_mmp_masks_rh.items():
    roi_mask_full_rh = np.zeros_like(full_brain_to_cortex_mask, dtype=bool)
    roi_mask_full_rh[rh_cortex_idx] = roi_mask
    rois_group_mmp_masks_full_rh_dict[roi_name] = roi_mask_full_rh

# --- Full brain ROIs ---
for roi_name, roi_mask in rois_group_mmp_masks_brain.items():
    roi_mask_full_brain = np.zeros_like(full_brain_to_cortex_mask, dtype=bool)
    roi_mask_full_brain[cortex_idx] = roi_mask
    rois_group_mmp_masks_full_brain_dict[roi_name] = roi_mask_full_brain



In [11]:
# Export masks as npz
np.savez('{}/{}_hemi-L_rois-group-mmp.npz'.format(roi_dir, full_brain_format), **rois_group_mmp_masks_full_lh_dict)
np.savez('{}/{}_hemi-R_rois-group-mmp.npz'.format(roi_dir, full_brain_format), **rois_group_mmp_masks_full_rh_dict)
np.savez('{}/{}_rois-group-mmp.npz'.format(roi_dir, full_brain_format), **rois_group_mmp_masks_full_brain_dict)